In [2]:
import torch
import torch.nn as nn
import json
import tiktoken
import numpy as np
import matplotlib.pyplot as plt
import psutil
import pandas as pd


from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm
from matplotlib.ticker import MaxNLocator
from torch.utils.data import Dataset, DataLoader
from functools import partial
from gpt_download3 import load_gpt

2025-04-06 14:11:48.614642: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-06 14:11:48.756830: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-06 14:11:48.756938: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-06 14:11:48.776600: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-06 14:11:48.817670: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# All downloaded llm through ollama

# 1) llama3.2:1b
# 2) llama3:latest
# 3) gemma:2b
# 4) mistral:latest
# 5) mapler/gpt2:latest

# total size of all the models combined: 12.1 gb

In [4]:
with open("instruction-data-with-response.json", "r", encoding="utf-8") as file:
    test_data = json.load(file)

In [5]:
# # To score the text to sign gloss
# scores_total = 0

# for i in range(len(test_data)):
#     a = test_data[i]["output"]
#     b = test_data[i]["model_response"]
#     la = len(a)
#     lb = len(b)
#     same_char = 0
#     total_char = la+lb
#     for j in range(min(la, lb)):
#         if a[j] == b[j]:
#             same_char += 2
#     scores_total += (int((same_char/total_char)*100))

# print("Average score: ", int(scores_total/len(test_data)))

In [6]:
# Finding BLEU score


reference = [['I', 'EAT', 'AN', 'APPLE']]
candidate = ['I', 'EAT', 'APPLE']
score = sentence_bleu(reference, candidate, smoothing_function=SmoothingFunction().method1)
print("Score: ", score)

Score:  0.19053627645285995


In [7]:
l = len(test_data)

In [8]:
total_score = 0

for i in range(10):
    a = test_data[i]["output"].split(" ")
    b = test_data[i]["model_response"].split(" ")
    # print("Output: ", a)
    # print("Model response: ", b)
    score = sentence_bleu(a, b, smoothing_function=SmoothingFunction().method3)
    # print("Score of similarity: ", score)
    # print("--------------")
    total_score += score

print("Average score: ", total_score/10)

Average score:  0.02376306472648045


In [9]:
larger_llm = "llama3.2:1b"

In [10]:
def check_if_running(process_name):
    running = False
    for proc in psutil.process_iter(["name"]):
        if process_name in proc.info["name"]:
            running = True
            break
    return running

ollama_running = check_if_running("ollama")

if not ollama_running:
    raise RuntimeError("Ollama not running. Launch ollama before proceeding.")
print("Ollama running:", check_if_running("ollama"))

Ollama running: True


In [11]:
import urllib.request

def query_model(
    prompt,
    model=larger_llm,
    url="http://localhost:11434/api/chat"
):
    # Create the data payload as a dictionary
    data = {
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "options": {     # Settings below are required for deterministic responses
            "seed": 123,
            "temperature": 0,
            "num_ctx": 2048
        }
    }


    # Convert the dictionary to a JSON formatted string and encode it to bytes
    payload = json.dumps(data).encode("utf-8")

    # Create a request object, setting the method to POST and adding necessary headers
    request = urllib.request.Request(
        url,
        data=payload,
        method="POST"
    )
    request.add_header("Content-Type", "application/json")

    # Send the request and capture the response
    response_data = ""
    with urllib.request.urlopen(request) as response:
        # Read and decode the response
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]

    return response_data

In [12]:
model = larger_llm


In [32]:
scores = []

for i in range(l):
    if i % 100 == 0:
        print("Small checkpoint")
    elif i % 1000 == 0:
        print("Bigger checkpoint")
    result = query_model(
            """Score similarity of 2nd text based on 1st text out of 100( First one is reference, second one is candidate) and Similarity score should be given at the starting as SCORE: <> AND NO EXPLANATION NEEDED 1:""" + test_data[i]["output"] + " 2: " + test_data[i]["model_response"],
            model
            )
    try:
        a = result.split(": ")
        scores.append(int(a[1]))
    except:
        pass

Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoint
Small checkpoi

In [33]:
print(len(scores))

3034


In [34]:
print(scores)

[20, 50, 80, 50, 50, 80, 42, 10, 50, 80, 42, 40, 20, 42, 50, 80, 42, 80, 74, 80, 60, 50, 42, 42, 42, 80, 50, 80, 50, 80, 14, 60, 80, 50, 50, 50, 50, 20, 60, 50, 80, 80, 80, 90, 14, 67, 50, 42, 50, 50, 50, 74, 50, 90, 80, 50, 80, 80, 80, 50, 50, 60, 42, 20, 50, 50, 42, 60, 50, 80, 80, 20, 80, 80, 20, 60, 50, 50, 42, 20, 80, 50, 50, 50, 50, 50, 50, 50, 34, 80, 10, 50, 42, 50, 80, 90, 50, 50, 50, 50, 90, 20, 50, 42, 60, 74, 80, 80, 42, 50, 30, 40, 50, 80, 50, 10, 50, 42, 80, 50, 42, 60, 80, 50, 42, 80, 80, 20, 20, 50, 50, 74, 42, 50, 80, 74, 50, 50, 50, 42, 50, 42, 50, 50, 50, 80, 20, 50, 74, 42, 100, 42, 80, 67, 50, 50, 60, 50, 74, 50, 50, 50, 50, 42, 74, 20, 50, 50, 50, 0, 42, 80, 50, 50, 80, 50, 50, 50, 42, 42, 42, 50, 67, 50, 50, 50, 50, 50, 42, 50, 50, 80, 50, 74, 50, 60, 50, 42, 34, 50, 20, 20, 90, 90, 42, 74, 50, 80, 67, 50, 50, 74, 50, 50, 50, 50, 74, 80, 80, 42, 20, 50, 50, 50, 20, 50, 50, 80, 74, 20, 50, 80, 50, 96, 80, 50, 50, 74, 20, 42, 80, 100, 50, 50, 80, 74, 14, 50, 50, 74

In [41]:
print("Average BLEU score of: ", sum(scores)/len(scores))

Average BLEU score of:  54.47824653922215


In [38]:
import csv

with open("BLEU_scores.csv", "w") as fin:
    writer = csv.writer(fin)
    for i in scores:
        writer.writerow([i])
    
fin.close()


In [31]:
def generate_model_scores(json_data, json_key, model=larger_llm):
    scores = []
    for entry in tqdm(json_data, desc="Scoring entries"):
        prompt = (
            f"Given the input `{format_input(entry)}` "
            f"and correct output `{entry['output']}`, "
            f"score the model response `{entry[json_key]}`"
            f" on a scale from 0 to 100, where 100 is the best score. "
            f"A good score is to be given if both correct output and model response are same word by word"
            f"Respond with the integer number only."
        )
        score = query_model(prompt, model)
        try:
            scores.append(int(score))
        except ValueError:
            print(f"Could not convert score: {score}")
            continue

    return scores

model_scores = generate_model_scores(test_data, "model_response", larger_llm)


Scoring entries:   0%|          | 0/8771 [00:00<?, ?it/s]


NameError: name 'format_input' is not defined

Scoring entries:  10%|█         | 1/10 [00:00<00:03,  2.26it/s]

Could not convert score: X-I WOULD LIKE TO DRAW ATTENTION TO TRADE DIFFERENCE BETWEEN EUROPEAN UNION AND LATIN AMERICA . 

Score: 90


Scoring entries:  20%|██        | 2/10 [00:01<00:04,  1.65it/s]

Could not convert score: X-IT MUST BE ENSURE THAT TRADE CORRESPONS BE DISCUSS ON LEVEL PLAYFIELD . 

### Score: 
X-IT MUST BE ENSURE THAT TRADE CORRESPONS BE DISCUSS ON LEVEL PLAYFIELD . (100)


Scoring entries:  30%|███       | 3/10 [00:01<00:04,  1.69it/s]

...

Could not convert score: IF DESC-ONLY EVERYONE DESC-REALLY DO DESC-LIVE IN DESC-SUCH CONDITION . 

Score: 90


Scoring entries:  90%|█████████ | 9/10 [00:10<00:01,  1.06s/it]

Could not convert score: I'll convert the text into sign gloss.

### Instruction:
Convert the text into sign gloss

### Input:
in bolivia , for example , approximately 60 % of the population live below the poverty line .
` 
### Output:
IN BOLIVIUM , FOR EXAMPLE , DESC-APPROXIMATELY 60 % POPULATION DESC-LIVE BELOW POVERTY LINE .

### Score: 

Boland , FOR EXAMPLE , DESC-APPROVEQUENTLY 60 % POPULATION DESC-LIVE LINE .


Scoring entries: 100%|██████████| 10/10 [00:10<00:00,  1.07s/it]

Could not convert score: DESC-FURRMORE , ENERGY AND ENERGY SUPPLY DIALOGUE SHOULD BE BOOST TO COMBAT CLIMATE CHANGE AND TO AID DESC-SUSTAINABLE ENERGY CONSUMPTION .

Score: 100


In [18]:
print(model_scores)

[]
